# Hidden Markov Models for Vietnamese POS Tagging

In this programming assignment, you will implement a first-order Hidden Markov Models for Vietnamese POS Tagging. We train the first-order HMM model on a training data and evaluate the accuracy of the model on a separate test data.

The due for the programming assignment is **March 12, 2020 (Hard deadline)**.

## Exercise 1: Write your name and student ID

Write your name and student id

- Name: Đỗ Đăng Minh Đức
- Student ID: USTHBI8-042

## Dataset

In the assignment, we will use the data obtained from Vietnamese Treebank corpus. The lecturer prepared the training/test data from the corpus.

We use 6000 labeled sentences for training and 1000 labeled for testing.

We will download two files [train.txt](https://www.dl.dropboxusercontent.com/s/xqkqmrkekb4uymv/train.txt) and [test.txt](https://www.dl.dropboxusercontent.com/s/4w98bbt4lapcdx2/test.txt)

In [0]:
!rm -f train.txt
!rm -f test.txt
!wget https://www.dl.dropboxusercontent.com/s/xqkqmrkekb4uymv/train.txt
!wget https://www.dl.dropboxusercontent.com/s/4w98bbt4lapcdx2/test.txt

Each line in a file is a tokenized sentence and each token is tagged with it part-of-speech.

In [0]:
!head -3 train.txt

Đó/P là/V con/Nc đường/N biển/N ngắn/A nhất/A để/E đi/V từ/E Ấn_Độ_Dương/NNP sang/V Thái_Bình_Dương/NNP ,/CH chiếm/V đến/E lượng/N hàng_hoá/N lưu_thông/V đường_biển/N của/E thế_giới/N ,/CH đó/P là/V hải_trình/N lớn/A nhất/R từ/E tây/N sang/V đông/N với/E 50.000/M lượt/Nc tàu_bè/N qua_lại/V mỗi/L năm/N .../CH
Một/M chuyến/N hải_trình/N xuyên/V ba/M nước/N Malaysia/NNP ,/CH Singapore/NNP ,/CH Indonesia/NNP vừa/R được/V phóng_viên/N Tuổi_Trẻ/NNP thực_hiện/V ,/CH để/E cảm_nhận/V điều/N mà/C các/L thuỷ_thủ/N tàu/N viễn_dương/A đã/R cảm_nhận/V mỗi/L khi/N nghe/V nhắc/V tới/R :/CH hải_tặc/N eo_biển/N Malacca/NNP !/CH .../CH
Từ/E bức/Nc điện/N của/E IMB/Ny .../CH


### Tagset

Let's figure out how many unique tags in the training data.

In [0]:
tagset = set()
with open('train.txt') as f:
    for line in f:
        line = line.rstrip()
        wordtags = line.split()
        for wordtag in wordtags:
            fields = wordtag.split('/')
            tag = fields[-1]
            if tag == 'chưa':
                print(wordtag)
            tagset.add(tag)

In [0]:
print('# Number of tags: %d' % len(tagset))

# Number of tags: 27


In [0]:
tagset

{'A',
 'C',
 'CH',
 'Cc',
 'E',
 'FW',
 'I',
 'L',
 'M',
 'N',
 'NNP',
 'Nb',
 'Nc',
 'Ne',
 'Ni',
 'Ns',
 'Nu',
 'Ny',
 'O',
 'P',
 'R',
 'T',
 'V',
 'Vb',
 'Vy',
 'X',
 'Z'}

Meaning of some Vietnamese part-of-speech tags is described as follows.

| Tag | Description | Example |
|-----|--------------|--------------|
| A   | Adjective   | rảnh_rỗi, đẹp |
| V   | Verb | kể |
| N   | Noun | tiền, vợ_con |

## Exercise 2: Training First-Order Hidden Markov Models for POS Tagging

Please refer to the pseudo code in the lecture slide. We implement the function `train` model that read the training file, calculate parameters of HMM model ans save parameters to a model file.

Parameters of our HMM model include transition probabilities and emission probabilities. Probabilities are calculated as follows.

Transition probabilities from POS $\rightarrow$ POS.

$$
P_T(t_i|t_{i-1})=\frac{C(t_{i-1}t_i)}{C(t_{i-1})}
$$

where $C(t_{i-1})$ is number of occurences of the tag $t_{i-1}$ in the data; and $C(t_{i-1}t_i)$ is the number of times that the tag $t_{i-1}$ followed by the tag $t_i$ in our training data.

Emission probabilities are calculated as follows.

$$
P_E(w_i|t_i)=\frac{C(t_i,w_i)}{C(t_i)}
$$

You will need to complete the function `train` by writing codes after comments with `# TODO:`.

In [0]:
from collections import defaultdict


def split_wordtag(wordtag):
    fields = wordtag.split('/')
    tag = fields.pop()
    word = '/'.join(fields)
    return word, tag


def train(train_file: str, model_file: str):
    emit = defaultdict(int)   # dictionary to store emission count C(t_i, w_i)
    transition = defaultdict(int)   # transition count C(t_{i-1}, t_i)
    context = defaultdict(int)  # count the context
    with open(train_file, 'r') as f:
        for line in f:
            line = line.strip()
            previous = '<s>'    # Make the sentence start
            context[previous] += 1
            wordtags = line.split()
            for wordtag in wordtags:
                word, tag = split_wordtag(wordtag)

                # TODO: Write code to update transition count, emission count
                # YOUR CODE HERE
                # ...

                transition[previous + " " + tag] +=1
                context[tag] +=1
                emit[tag + " " + word] +=1
                previous = tag


                # END OF YOUR CODE

            transition[previous + ' </s>'] += 1   # Sentence stop

    # Now we will save the parameters of the model to a file
    with open(model_file, 'w') as fo:

        # Save transition probabilities
        for key, value in transition.items():
            previous, word = key.split(' ')
            fo.write('T %s %f\n' % (key, value/context[previous]))

        # TODO: Write emission probabilities into the model file
        # YOUR CODE HERE

        for k, val in emit.items():
            tag, word = k.split(' ')
            emit_prob = emit[k] / context[tag]

        # Save emission probabilities
        
            fo.write('E %s %f\n' % (k, emit_prob))
      

        # END OF YOUR CODE
    print('Finished training first-order HMM!')

In the `train` function, we will use dict `emit` to store count values $C(t_i, w_i)$ and `transition` to store count values $C(t_{i-1}t_i)$.  **The keys of dictionaries are built by joining two strings (two POS tags or a pair of (tag, word) with a single space**.

In the model file, we save each probability in a line. In the beginning of each line, we write "T" to indicate transition probabilities and "E" for emission probabilities.

The content of the model file is something like as follows.

```
T <s> P 0.079667
T P V 0.316036
T V Nc 0.007959
T Nc N 0.829887
T N N 0.182656
...
E V liên_kế 0.000038
E N xế 0.000034
E A rề_rà 0.000139
E A quanh 0.000139
E V túa 0.000038
```

Let's call the train function.

In [0]:
train('train.txt', 'HMM_model.txt')

Let's see some first lines of the model.

In [0]:
!head HMM_model.txt

In [0]:
!tail HMM_model.txt

## Exercise 3: Loading HMM model from file

In the exercise 3, you need to write code to load the model file which we trained above into three dictionaries:

- `transition` is the dictionary to store transition probabilities $P_T(t_i|t_{i-1})$.
- `emission` is the dictionary to store emission probabilities $P_E(w_i|t_i)$.
- `possible_tags` is the dictionary to store unique tags in the model. It maps from a POS tag to 1.

Hint: You need to check if the line starting with 'T' or 'E' to load transition and emission probabilities.

In [0]:
def load_model(model_file: str):
    """Load saved HMM model
    """
    transition = defaultdict(lambda: 0)
    emission = defaultdict(lambda: 0)
    possible_tags = {}
    
    # TODO: Write your code to load the model file
    # YOUR CODE HERE

    with open(model_file, 'r') as f:
      for line in f:
      
        line = line.strip()
        prob_type, tag1, t2, prob = line.split(" ")

        if (prob_type == 'T'): transition[tag1 + " " + t2] = prob
        else :
          emission[tag1 + " " + t2] = prob
        
        possible_tags[tag1] = 1
    
    
    # END OF YOUR CODE HERE
    return transition, emission, possible_tags

Let's try to call the function and see the content of results.

In [0]:
transition, emission, possible_tags = load_model('HMM_model.txt')
print( list(possible_tags.keys()) )
print(transition)
print(emission)

['<s>', 'P', 'V', 'Nc', 'N', 'A', 'E', 'NNP', 'CH', 'R', 'M', 'L', 'C', 'Ny', 'T', 'Nb', 'Ns', 'Nu', 'Cc', 'FW', 'Ne', 'Vb', 'I', 'X', 'Z', 'Ni', 'O', 'Vy']
defaultdict(<function load_model.<locals>.<lambda> at 0x7f694a80fe18>, {'<s> P': 0.079667, 'P V': 0.316036, 'V Nc': 0.007959, 'Nc N': 0.829887, 'N N': 0.182656, 'N A': 0.084451, 'A A': 0.064919, 'A E': 0.117492, 'E V': 0.077012, 'V E': 0.09452, 'E NNP': 0.084095, 'NNP V': 0.248928, 'V NNP': 0.020983, 'NNP CH': 0.295799, 'CH V': 0.111818, 'E N': 0.493496, 'N V': 0.205183, 'V N': 0.27442, 'N E': 0.076728, 'N CH': 0.16836, 'CH P': 0.031586, 'A R': 0.063809, 'R E': 0.023335, 'E M': 0.047392, 'M Nc': 0.043661, 'V L': 0.023078, 'L N': 0.742995, 'CH </s>': 0.337425, '<s> M': 0.040333, 'M N': 0.482606, 'V M': 0.03534, 'N NNP': 0.058638, 'CH NNP': 0.041327, 'NNP R': 0.103744, 'R V': 0.558338, 'V CH': 0.120606, 'CH E': 0.029278, 'N C': 0.020867, 'C L': 0.037297, 'V V': 0.211318, 'V R': 0.054648, 'R CH': 0.064264, 'CH N': 0.146163, 'CH CH': 0

## Exercise 4: Viterbi algorithm

In the exercise 4, you will implement the viterbi algorithm to determine the tag sequence for a tokenized sentence given the probabilities we have loaded from the model file.

In [0]:
import math


def viterbi(line, transition, emission, possible_tags):
    """Infer the tag sequence for a tokenized sentence

    Args:
        line (str): a tokenized word sequence
                    e.g., "Chiều cuối thu , trời vùng_biển Nghi_Xuân ảm_đạm ."
        transition (dict): transition probabilities
        emission (dict): emission probabilities
    """
    words = line.split()
    l = len(words)
    best_score = {}
    best_edge = {}
    best_score[('0 <s>')] = 0  # Start with <s>
    best_edge[('0 <s>')] = None
    # Forward Step
    for i in range(l):
        for prev in possible_tags.keys():
            for _next in possible_tags.keys():
                if str(i) + ' ' + prev in best_score and prev + ' ' + _next in transition:
                    if emission[_next + ' ' + words[i]] == 0:
                        # To avoid zero probabilities, we use very small value
                        emission[_next + " " + words[i]] = 10 ** (-10)
                    
                    # TODO: Write code to calculate the score of the path that
                    # connect the best path in the step i with the edge prev -> _next
                    # score = ...
                    
                    score = best_score[str(i) + ' ' + prev] + -math.log(float(transition[prev + ' ' + _next])) + -math.log(float(emission[_next + ' ' + words[i]]))

                    # END OF YOUR CODE HERE
                    if str(i + 1) + " " + _next not in best_score or best_score[str(i + 1) + " " + _next] > score:
                        best_score[str(i + 1) + " " + _next] = score
                        best_edge[str(i + 1) + " " + _next] = str(i) + " " + prev
    
    for prev in possible_tags.keys():
        if str(l) + ' ' + prev in best_score:
            if (prev + ' ' + '</s>') not in transition:
                transition[prev + ' ' + '</s>'] = 10 ** (-10)
            
            # TODO: Calculate best_score[str(l+1) + ' </s>'] and best_edge[str(l+1) + ' </s>'] 
            # for the sentence top symbole '</s>'
            # The different from the other time step is that, we do not use emission probility in calculating score
            # YOUR CODE HERE

            score = best_score[str(l) + ' ' + prev] + -math.log(float(transition[prev + ' ' + '</s>']))
            if str(l + 1) + ' ' + '</s>' not in best_score or best_score[str(l + 1) + ' ' + '</s>'] > score:
              best_score[str(l + 1) + ' ' + '</s>'] = score
              best_edge[str(l + 1) + ' ' + '</s>'] = str(l) + ' ' + prev

    
            # END OF YOUR CODE
    
    # Backward Step
    tags = []
    next_edge = best_edge[str(l + 1) + " " + "</s>"]
    # TODO: Complete the backward step in Viterbi algorithm
    # Finish the while loop in the pseudo code

    while (next_edge !="0 <s>"):
      position,tag = next_edge.split(' ')
      tags.append(tag)
      next_edge = best_edge[next_edge]
    
    # END OF YOUR CODE
    tags.reverse()
    return ' '.join(tags)

Let's try to use the function viterbi function to find the tag sequence for a given sentence.

In [0]:
viterbi('Chiều cuối thu , trời vùng_biển Nghi_Xuân ảm_đạm .', transition, emission, possible_tags)

'N N V CH N N NNP A CH'

You should get the tag sequence, let\'s say \'N N V CH N N NNP A CH\'.

## Exercise 5: Model Evaluation

In the exercise, you do not to write code. Just understand and execute the code.

We will evaluate the model on the test data with accuracy measure.

In [0]:
def split_wordtag(wordtag):
    fields = wordtag.split('/')
    tag = fields.pop()
    word = '/'.join(fields)
    return word, tag


def evaluate(test_file: str, transition, emission, possible_tags):
    correct = 0
    total = 0

    with open(test_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            wordtags = line.split()
            words = []
            gold_tags = []
            for wordtag in wordtags:
                word, tag = split_wordtag(wordtag)
                words.append(word)
                gold_tags.append(tag)
            sentence = ' '.join(words)
            predicted_tags = viterbi(sentence, transition, emission, possible_tags)
            predicted_tags = predicted_tags.split(' ')
            for t1, t2 in zip(predicted_tags, gold_tags):
                total += 1
                if t1 == t2:
                    correct += 1
    return 100.0 * correct/total, correct, total

In [0]:
acc, correct, total = evaluate('test.txt', transition, emission, possible_tags)
print("Accuracy = {} ({}/{})".format(acc, correct, total))

Accuracy = 86.94539068616042 (41093/47263)


You should get the accuracy about 86.9%.

## Bonus exercises

- You will get bonus points if you can use some advanced techniques for dealing with unknown words or adding more features into the HMM model.
- You will get bonus points if you can obtain better accuracy than 87% using HMM model. For instance, you can implement the second-order HMM tagger.